<a href="https://colab.research.google.com/github/MehrdadJalali-AI/LotusEffectAlgorithm/blob/main/MLEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import gamma

# Define the 14 benchmark functions
def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

def six_hump(x):
    x1, x2 = x[0], x[1]
    return (4 - 2.1 * x1**2 + (x1**4)/3) * x1**2 + x1*x2 + (-4 + 4*x2**2) * x2**2

def decreasing_maxima(x):
    x = np.maximum(x, 1e-10)  # Avoid invalid power operations
    return np.exp(-2 * np.log(2) * ((x - 0.08)/0.854)**2) * np.sin(5 * np.pi * ((x**(3/4)) - 0.05))**6

def equal_maxima(x):
    return np.sin(5 * np.pi * x)**6

def himmelblau(x):
    x1, x2 = x[0], x[1]
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2

def modified_rastrigin(x):
    return np.sum(10 + 9 * np.cos(2 * np.pi * (x - 0.5)))

def vincent(x):
    x = np.maximum(x, 1e-10)  # Avoid log(0) errors
    return (1 / len(x)) * np.sum(np.sin(10 * np.log(x)))

def shubert(x):
    x1, x2 = x[0], x[1]
    return np.prod([np.sum(i * np.cos((i + 1) * xj + i)) for i in range(1, 6) for xj in [x1, x2]])

def composition_function_1(x):
    return np.sum(x**2) + np.sum(np.cos(2 * np.pi * x))

def composition_function_2(x):
    return np.sum(x**2) + np.sum(np.sin(2 * np.pi * x))

def five_uneven_peak_trap(x):
    return np.sin(5 * np.pi * x)**6

def expanded_equal_maxima(x):
    return np.sum(np.sin(5 * np.pi * x)**6)

def expanded_uneven_maxima(x):
    return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)

def modified_vincent(x):
    return np.sum(np.sin(10 * np.log(x)) + 1)

# Store all functions in a dictionary
benchmark_functions = {
    "Ackley": ackley,
    "Six Hump": six_hump,
    "Decreasing Maxima": decreasing_maxima,
    "Equal Maxima": equal_maxima,
    "Himmelblau": himmelblau,
    "Modified Rastrigin": modified_rastrigin,
    "Vincent": vincent,
    "Shubert": shubert,
    "Composition Function 1": composition_function_1,
    "Composition Function 2": composition_function_2,
    "Five Uneven Peak Trap": five_uneven_peak_trap,
    "Expanded Equal Maxima": expanded_equal_maxima,
    "Expanded Uneven Maxima": expanded_uneven_maxima,
    "Modified Vincent": modified_vincent
}

# Define the Lotus Effect Algorithm (LEA) with MBRO-inspired enhancements
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_function_evaluations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_function_evaluations = max_function_evaluations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.best_solution = self.population[0].copy()
        self.best_fitness = float('inf')
        self.function_evaluations = 0
        self.alpha = 0.5
        self.beta = 1.5
        self.memory = []

    def levy_flight(self):
        sigma = (gamma(1 + self.beta) * np.sin(np.pi * self.beta / 2) /
                 (gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))) ** (1 / self.beta)
        u = np.random.normal(0, sigma, size=self.dimensions)
        v = np.random.normal(0, 1, size=self.dimensions)
        step = u / np.abs(v) ** (1 / self.beta)
        return 0.01 * step

    def update_positions(self):
        for i in range(self.population_size):
            if self.function_evaluations >= self.max_function_evaluations:
                return
            step = self.alpha * (self.best_solution - self.population[i]) + self.levy_flight()
            self.population[i] += step
            self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
            self.function_evaluations += 1

    def evaluate_fitness(self):
        for i in range(self.population_size):
            if self.function_evaluations >= self.max_function_evaluations:
                return
            fitness = self.fitness_function(self.population[i])
            self.function_evaluations += 1
            if np.isscalar(fitness):
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()
                    self.memory.append(self.best_solution)
            else:
                if np.all(fitness < self.best_fitness):
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()
                    self.memory.append(self.best_solution)

    def optimize(self):
        while self.function_evaluations < self.max_function_evaluations:
            self.update_positions()
            self.evaluate_fitness()
        return self.best_solution, self.best_fitness, self.function_evaluations

# Running evaluations for all 14 functions
num_runs = 10
max_function_evaluations = 100000

evaluation_counts = []
optimal_values = []
success_rates = []
success_performance = []
maximum_peak_ratios = []

for name, func in benchmark_functions.items():
    best_fitnesses = []
    function_evals = []
    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(
            population_size=30, dimensions=2, lower_bound=-5, upper_bound=5,
            max_function_evaluations=max_function_evaluations, fitness_function=func
        )
        _, best_fitness, function_evaluations = lea.optimize()
        best_fitnesses.append(best_fitness)
        function_evals.append(function_evaluations)

    evaluation_counts.append(np.mean(function_evals))
    optimal_values.append(np.mean(best_fitnesses))
    success_rate = np.mean(np.array(best_fitnesses) <= 0.01)
    success_rates.append(success_rate)
    success_performance.append(np.sum(function_evals) / (num_runs * success_rates[-1]) if success_rates[-1] > 0 else np.inf)
    maximum_peak_ratios.append(np.mean(best_fitnesses))

# Create results table
df_results = pd.DataFrame({
    "Function": list(benchmark_functions.keys()),
    "Average Function Evaluations": evaluation_counts,
    "LEA Average Best Fitness": optimal_values,
    "Success Rate (SR)": success_rates,
    "Success Performance (SP)": success_performance,
    "Maximum Peak Ratio (MPR)": maximum_peak_ratios
})

# Save results to CSV
df_results.to_csv("LEA_Results.csv", index=False)

# Display the results table
print(df_results)


<ipython-input-5-db070f94ad1c>:50: RuntimeWarning: invalid value encountered in power
  return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)
<ipython-input-5-db070f94ad1c>:53: RuntimeWarning: invalid value encountered in log
  return np.sum(np.sin(10 * np.log(x)) + 1)


                  Function  Average Function Evaluations  \
0                   Ackley                      100000.0   
1                 Six Hump                      100000.0   
2        Decreasing Maxima                      100000.0   
3             Equal Maxima                      100000.0   
4               Himmelblau                      100000.0   
5       Modified Rastrigin                      100000.0   
6                  Vincent                      100000.0   
7                  Shubert                      100000.0   
8   Composition Function 1                      100000.0   
9   Composition Function 2                      100000.0   
10   Five Uneven Peak Trap                      100000.0   
11   Expanded Equal Maxima                      100000.0   
12  Expanded Uneven Maxima                      100000.0   
13        Modified Vincent                      100000.0   

    LEA Average Best Fitness  Success Rate (SR)  Success Performance (SP)  \
0               1.6194

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import gamma

# Define the 14 benchmark functions
def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

def six_hump(x):
    x1, x2 = x[0], x[1]
    return (4 - 2.1 * x1**2 + (x1**4)/3) * x1**2 + x1*x2 + (-4 + 4*x2**2) * x2**2

def decreasing_maxima(x):
    x = np.maximum(x, 1e-10)  # Avoid invalid power operations
    return np.exp(-2 * np.log(2) * ((x - 0.08)/0.854)**2) * np.sin(5 * np.pi * ((x**(3/4)) - 0.05))**6

def equal_maxima(x):
    return np.sin(5 * np.pi * x)**6

def himmelblau(x):
    x1, x2 = x[0], x[1]
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2

def modified_rastrigin(x):
    return np.sum(10 + 9 * np.cos(2 * np.pi * (x - 0.5)))

def vincent(x):
    x = np.maximum(x, 1e-10)  # Avoid log(0) errors
    return (1 / len(x)) * np.sum(np.sin(10 * np.log(x)))

def shubert(x):
    x1, x2 = x[0], x[1]
    return np.prod([np.sum(i * np.cos((i + 1) * xj + i)) for i in range(1, 6) for xj in [x1, x2]])

def composition_function_1(x):
    return np.sum(x**2) + np.sum(np.cos(2 * np.pi * x))

def composition_function_2(x):
    return np.sum(x**2) + np.sum(np.sin(2 * np.pi * x))

def five_uneven_peak_trap(x):
    return np.sin(5 * np.pi * x)**6

def expanded_equal_maxima(x):
    return np.sum(np.sin(5 * np.pi * x)**6)

def expanded_uneven_maxima(x):
    return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)

def modified_vincent(x):
    return np.sum(np.sin(10 * np.log(x)) + 1)

# Store all functions in a dictionary
benchmark_functions = {
    "Ackley": ackley,
    "Six Hump": six_hump,
    "Decreasing Maxima": decreasing_maxima,
    "Equal Maxima": equal_maxima,
    "Himmelblau": himmelblau,
    "Modified Rastrigin": modified_rastrigin,
    "Vincent": vincent,
    "Shubert": shubert,
    "Composition Function 1": composition_function_1,
    "Composition Function 2": composition_function_2,
    "Five Uneven Peak Trap": five_uneven_peak_trap,
    "Expanded Equal Maxima": expanded_equal_maxima,
    "Expanded Uneven Maxima": expanded_uneven_maxima,
    "Modified Vincent": modified_vincent
}

# Define the Lotus Effect Algorithm (LEA) with MBRO-inspired enhancements
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_function_evaluations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_function_evaluations = max_function_evaluations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.best_solution = self.population[0].copy()
        self.best_fitness = float('inf')
        self.function_evaluations = 0
        self.alpha = 0.5
        self.beta = 1.5
        self.memory = []

    def levy_flight(self):
        sigma = (gamma(1 + self.beta) * np.sin(np.pi * self.beta / 2) /
                 (gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))) ** (1 / self.beta)
        u = np.random.normal(0, sigma, size=self.dimensions)
        v = np.random.normal(0, 1, size=self.dimensions)
        step = u / np.abs(v) ** (1 / self.beta)
        return 0.01 * step

    def update_positions(self):
        for i in range(self.population_size):
            if self.function_evaluations >= self.max_function_evaluations:
                return
            step = self.alpha * (self.best_solution - self.population[i]) + self.levy_flight()
            self.population[i] += step
            self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
            self.function_evaluations += 1

    def evaluate_fitness(self):
        for i in range(self.population_size):
            if self.function_evaluations >= self.max_function_evaluations:
                return
            fitness = self.fitness_function(self.population[i])
            self.function_evaluations += 1
            if np.isscalar(fitness):
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()
                    self.memory.append(self.best_solution)
            else:
                if np.all(fitness < self.best_fitness):
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()
                    self.memory.append(self.best_solution)

    def optimize(self):
        while self.function_evaluations < self.max_function_evaluations:
            self.update_positions()
            self.evaluate_fitness()
        return self.best_solution, self.best_fitness, self.function_evaluations

# Running evaluations for all 14 functions
num_runs = 10
max_function_evaluations = 100000

evaluation_counts = []
optimal_values = []
success_rates = []
success_performance = []
maximum_peak_ratios = []



global_optima = {name: [1.0] * num_runs for name in benchmark_functions.keys()}  # Placeholder for actual global optima

for name, func in benchmark_functions.items():
    best_fitnesses = []
    function_evals = []
    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(
            population_size=30, dimensions=2, lower_bound=-5, upper_bound=5,
            max_function_evaluations=max_function_evaluations, fitness_function=func
        )
        _, best_fitness, function_evaluations = lea.optimize()
        best_fitnesses.append(best_fitness)
        function_evals.append(function_evaluations)

    evaluation_counts.append(np.mean(function_evals))
    optimal_values.append(np.mean(best_fitnesses))

    # ✅ Success Rate (SR)
    success_rate = np.mean(np.array(best_fitnesses) <= 0.01)
    success_rates.append(success_rate)

    # ✅ Success Performance (SP)
    success_performance.append(
        np.sum(function_evals) / (num_runs * success_rates[-1]) if success_rates[-1] > 0 else np.inf
    )

    # ✅ Fixed Maximum Peak Ratio (MPR)
    total_found_peaks = np.sum(np.abs(best_fitnesses))
    total_actual_peaks = np.sum(np.abs(global_optima[name]))

    # ✅ Avoid division by zero & keep within [0,1] range
    if total_actual_peaks > 0:
        MPR_value = np.clip(total_found_peaks / total_actual_peaks, 0, 1)
    else:
        MPR_value = 0

    maximum_peak_ratios.append(MPR_value)


# Create results table
df_results = pd.DataFrame({
    "Function": list(benchmark_functions.keys()),
    "Average Function Evaluations": evaluation_counts,
    "LEA Average Best Fitness": optimal_values,
    "Success Rate (SR)": success_rates,
    "Success Performance (SP)": success_performance,
    "Maximum Peak Ratio (MPR)": maximum_peak_ratios
})

# Save results
df_results.to_csv("LEA_Results_StopWhenReach.csv", index=False)

# Display results
print(df_results)


<ipython-input-7-a59a6a6b6bfb>:50: RuntimeWarning: invalid value encountered in power
  return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)
<ipython-input-7-a59a6a6b6bfb>:53: RuntimeWarning: invalid value encountered in log
  return np.sum(np.sin(10 * np.log(x)) + 1)


                  Function  Average Function Evaluations  \
0                   Ackley                      100000.0   
1                 Six Hump                      100000.0   
2        Decreasing Maxima                      100000.0   
3             Equal Maxima                      100000.0   
4               Himmelblau                      100000.0   
5       Modified Rastrigin                      100000.0   
6                  Vincent                      100000.0   
7                  Shubert                      100000.0   
8   Composition Function 1                      100000.0   
9   Composition Function 2                      100000.0   
10   Five Uneven Peak Trap                      100000.0   
11   Expanded Equal Maxima                      100000.0   
12  Expanded Uneven Maxima                      100000.0   
13        Modified Vincent                      100000.0   

    LEA Average Best Fitness  Success Rate (SR)  Success Performance (SP)  \
0               2.0576

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import gamma

# Define the 14 benchmark functions
def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

def six_hump(x):
    x1, x2 = x[0], x[1]
    return (4 - 2.1 * x1**2 + (x1**4)/3) * x1**2 + x1*x2 + (-4 + 4*x2**2) * x2**2

def decreasing_maxima(x):
    x = np.maximum(x, 1e-10)
    return np.exp(-2 * np.log(2) * ((x - 0.08)/0.854)**2) * np.sin(5 * np.pi * ((x**(3/4)) - 0.05))**6

def equal_maxima(x):
    return np.sin(5 * np.pi * x)**6

def himmelblau(x):
    x1, x2 = x[0], x[1]
    return (x1**2 + x2 - 11)**2 + (x1 + x2**2 - 7)**2

def modified_rastrigin(x):
    return np.sum(10 + 9 * np.cos(2 * np.pi * (x - 0.5)))

def vincent(x):
    x = np.maximum(x, 1e-10)
    return (1 / len(x)) * np.sum(np.sin(10 * np.log(x)))

def shubert(x):
    x1, x2 = x[0], x[1]
    return np.prod([np.sum(i * np.cos((i + 1) * xj + i)) for i in range(1, 6) for xj in [x1, x2]])

def composition_function_1(x):
    return np.sum(x**2) + np.sum(np.cos(2 * np.pi * x))

def composition_function_2(x):
    return np.sum(x**2) + np.sum(np.sin(2 * np.pi * x))

def five_uneven_peak_trap(x):
    return np.sin(5 * np.pi * x)**6

def expanded_equal_maxima(x):
    return np.sum(np.sin(5 * np.pi * x)**6)

def expanded_uneven_maxima(x):
    return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)

def modified_vincent(x):
    return np.sum(np.sin(10 * np.log(x)) + 1)

# Store all functions in a dictionary
benchmark_functions = {
    "Ackley": ackley,
    "Six Hump": six_hump,
    "Decreasing Maxima": decreasing_maxima,
    "Equal Maxima": equal_maxima,
    "Himmelblau": himmelblau,
    "Modified Rastrigin": modified_rastrigin,
    "Vincent": vincent,
    "Shubert": shubert,
    "Composition Function 1": composition_function_1,
    "Composition Function 2": composition_function_2,
    "Five Uneven Peak Trap": five_uneven_peak_trap,
    "Expanded Equal Maxima": expanded_equal_maxima,
    "Expanded Uneven Maxima": expanded_uneven_maxima,
    "Modified Vincent": modified_vincent
}

# Number of local and global optima for each function
optima_counts = {
    "Ackley": (120, 1),
    "Six Hump": (4, 2),
    "Decreasing Maxima": (4, 1),
    "Equal Maxima": (0, 5),
    "Himmelblau": (0, 4),
    "Modified Rastrigin": (0, 12),
    "Vincent": (0, 36),
    "Shubert": (742, 18),
    "Composition Function 1": (119, 6),
    "Composition Function 2": (120, 6),
    "Five Uneven Peak Trap": (21, 4),
    "Expanded Equal Maxima": (0, 25),
    "Expanded Uneven Maxima": (0, 25),
    "Modified Vincent": (0, 36)
}

# Define actual global optima (as extracted from Table 1 of PaperForCompare.pdf)
global_optima = {
    "Ackley": 0.0,
    "Six Hump": -1.0316,
    "Decreasing Maxima": 0.8,
    "Equal Maxima": 1.0,
    "Himmelblau": 0.0,
    "Modified Rastrigin": 0.0,
    "Vincent": -1.0,
    "Shubert": -186.7309,
    "Composition Function 1": 0.0,
    "Composition Function 2": 0.0,
    "Five Uneven Peak Trap": 200.0,
    "Expanded Equal Maxima": 1.0,
    "Expanded Uneven Maxima": 1.0,
    "Modified Vincent": -1.0,
}


# Define the LEA Algorithm
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_function_evaluations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_function_evaluations = max_function_evaluations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.best_solution = self.population[0].copy()
        self.best_fitness = float('inf')
        self.function_evaluations = 0

    def optimize(self):
        while self.function_evaluations < self.max_function_evaluations:
            for i in range(self.population_size):
                if self.function_evaluations >= self.max_function_evaluations:
                    return self.best_solution, self.best_fitness, self.function_evaluations

                fitness = self.fitness_function(self.population[i])
                self.function_evaluations += 1

                # ✅ Ensure fitness is a scalar value
                if isinstance(fitness, np.ndarray):
                    fitness = np.min(fitness)

                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_solution = self.population[i].copy()

        return self.best_solution, self.best_fitness, self.function_evaluations

# Running evaluations
num_runs = 10
max_function_evaluations = 100000

evaluation_counts = []
optimal_values = []
success_rates = []
success_performance = []
maximum_peak_ratios = []

for name, func in benchmark_functions.items():
    best_fitnesses = []
    function_evals = []

    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(
            population_size=30, dimensions=2, lower_bound=-5, upper_bound=5,
            max_function_evaluations=max_function_evaluations, fitness_function=func
        )
        _, best_fitness, function_evaluations = lea.optimize()
        best_fitnesses.append(best_fitness)
        function_evals.append(function_evaluations)

    evaluation_counts.append(np.mean(function_evals))
    optimal_values.append(np.mean(best_fitnesses))

    success_rates.append(np.mean(np.array(best_fitnesses) <= 0.01))
    success_performance.append(np.sum(function_evals) / (num_runs * success_rates[-1]) if success_rates[-1] > 0 else np.inf)

    actual_optimum = global_optima[name]
    best_fitnesses_array = np.array(best_fitnesses)
    maximum_peak_ratios.append(np.clip(np.mean(np.abs(best_fitnesses_array - actual_optimum)) / np.abs(actual_optimum), 0, 1))

# Save results
df_results = pd.DataFrame({
    "Function": list(benchmark_functions.keys()),
    "Local Optima": [optima_counts[name][0] for name in benchmark_functions.keys()],
    "Global Optima": [optima_counts[name][1] for name in benchmark_functions.keys()],
    "Average Function Evaluations": evaluation_counts,
    "LEA Average Best Fitness": optimal_values,
    "Success Rate (SR)": success_rates,
    "Success Performance (SP)": success_performance,
    "Maximum Peak Ratio (MPR)": maximum_peak_ratios
})


df_results.to_csv("LEA_Results.csv", index=False)
print(df_results)


<ipython-input-2-2976d5f87156>:174: RuntimeWarning: divide by zero encountered in scalar divide
  maximum_peak_ratios.append(np.clip(np.mean(np.abs(best_fitnesses_array - actual_optimum)) / np.abs(actual_optimum), 0, 1))
<ipython-input-2-2976d5f87156>:50: RuntimeWarning: invalid value encountered in power
  return np.sum(np.sin(5 * np.pi * (x**(3/4) - 0.05))**6)
<ipython-input-2-2976d5f87156>:53: RuntimeWarning: invalid value encountered in log
  return np.sum(np.sin(10 * np.log(x)) + 1)


                  Function  Local Optima  Global Optima  \
0                   Ackley           120              1   
1                 Six Hump             4              2   
2        Decreasing Maxima             4              1   
3             Equal Maxima             0              5   
4               Himmelblau             0              4   
5       Modified Rastrigin             0             12   
6                  Vincent             0             36   
7                  Shubert           742             18   
8   Composition Function 1           119              6   
9   Composition Function 2           120              6   
10   Five Uneven Peak Trap            21              4   
11   Expanded Equal Maxima             0             25   
12  Expanded Uneven Maxima             0             25   
13        Modified Vincent             0             36   

    Average Function Evaluations  LEA Average Best Fitness  Success Rate (SR)  \
0                       100000.0 

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import gamma

# Define Levy Flight Function
def levy_flight(d, beta=1.5):
    sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) /
             (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
    u = np.random.randn(1, d) * sigma
    v = np.random.randn(1, d)
    step = u / (np.abs(v) ** (1 / beta))
    return 0.01 * step.flatten()

# Define the benchmark functions
def storn_chebyshev(x):
    n = len(x)
    return sum([(np.cos((j + 1) * np.arccos(x[i])) - (-1) ** j) ** 2 for i in range(n) for j in range(n)])

def inverse_hilbert(x):
    n = len(x)
    H = np.array([[1 / (i + j + 1) for j in range(n)] for i in range(n)])
    return np.linalg.norm(np.linalg.inv(H) - np.diag(x))

def lennard_jones(x, epsilon=1e-9):
    n = len(x) // 3
    X, Y, Z = x[:n], x[n:2*n], x[2*n:]
    energy = 0
    for i in range(n):
        for j in range(i + 1, n):
            r2 = (X[i] - X[j])**2 + (Y[i] - Y[j])**2 + (Z[i] - Z[j])**2
            if r2 < epsilon:
                continue  # Avoid division by zero
            energy += 4 * ((1/r2)**6 - (1/r2)**3)
    return energy

def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

def griewank(x):
    sum_part = sum(xi**2 / 4000 for xi in x)
    prod_part = np.prod([np.cos(xi / np.sqrt(i + 1)) for i, xi in enumerate(x)])
    return sum_part - prod_part + 1

def weierstrass(x, a=0.5, b=3, k_max=20):
    return sum([sum([a**k * np.cos(2 * np.pi * b**k * (xi + 0.5)) for k in range(k_max)]) for xi in x])

def modified_schwefel(x):
    return 418.9829 * len(x) - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

def expanded_schaffer_f6(x):
    def f6(xi, xj):
        return 0.5 + (np.sin(np.sqrt(xi**2 + xj**2))**2 - 0.5) / (1 + 0.001 * (xi**2 + xj**2))**2
    return sum([f6(x[i], x[i+1]) for i in range(len(x)-1)])

def happy_cat(x):
    alpha = 1/8
    sum_x2 = sum([xi**2 for xi in x])
    return ((sum_x2 - len(x))**2)**alpha + (0.5 * sum_x2 + sum(x)) / len(x) + 0.5

def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

# Function details with search range and dimensions
function_details = {
    "Storn’s Chebyshev": (9, [-8192, 8192]),
    "Inverse Hilbert": (16, [-16384, 16384]),
    "Lennard-Jones": (18, [-4, 4]),
    "Rastrigin": (10, [-100, 100]),
    "Griewank": (10, [-100, 100]),
    "Weierstrass": (10, [-100, 100]),
    "Modified Schwefel": (10, [-100, 100]),
    "Expanded Schaffer F6": (10, [-100, 100]),
    "Happy Cat": (10, [-100, 100]),
    "Ackley": (10, [-100, 100])
}

benchmark_functions = {
    "Storn’s Chebyshev": storn_chebyshev,
    "Inverse Hilbert": inverse_hilbert,
    "Lennard-Jones": lennard_jones,
    "Rastrigin": rastrigin,
    "Griewank": griewank,
    "Weierstrass": weierstrass,
    "Modified Schwefel": modified_schwefel,
    "Expanded Schaffer F6": expanded_schaffer_f6,
    "Happy Cat": happy_cat,
    "Ackley": ackley
}

# Global Optima for Success Rate Calculation
global_optima = {
    "Storn’s Chebyshev": 0.0,
    "Inverse Hilbert": 0.0,
    "Lennard-Jones": -1.0,
    "Rastrigin": 0.0,
    "Griewank": 0.0,
    "Weierstrass": 0.0,
    "Modified Schwefel": 0.0,
    "Expanded Schaffer F6": 0.0,
    "Happy Cat": 0.0,
    "Ackley": 0.0
}

# Define the LEA Algorithm
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_iterations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_iterations = max_iterations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))

    def optimize(self):
        best_fitnesses = []
        function_evaluations = 0

        for _ in range(self.max_iterations):
            for i in range(self.population_size):
                if np.random.rand() < 0.5:
                    self.population[i] += levy_flight(self.dimensions) * self.population[i]

                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
                fitness = self.fitness_function(self.population[i])
                best_fitnesses.append(fitness)
                function_evaluations += 1

        return np.min(best_fitnesses), function_evaluations

# Run LEA Optimization and Collect Metrics
num_runs = 10
max_iterations = 500

metrics = {
    "Function": [],
    "Local Optima": [],
    "Global Optima": [],
    "Average Function Evaluations": [],
    "LEA Average Best Fitness": [],
    "Success Rate (SR)": [],
    "Success Performance (SP)": [],
    "Maximum Peak Ratio (MPR)": [],
    "Peak Coverage (PC)": [],
    "Robustness Score": [],
    "Population Diversity (PD)": [],
    "Success Rate with Threshold (SRT)": [],
    "Convergence Speed (CS)": []
}

for name, func in benchmark_functions.items():
    best_fitnesses, function_evals, all_solutions = [], [], []

    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(30, 10, -100, 100, max_iterations, func)
        best_fitness, function_evaluations = lea.optimize()
        best_fitnesses.append(best_fitness)
        function_evals.append(function_evaluations)
        all_solutions.append(best_fitness)

    mean_fitness = np.mean(best_fitnesses)
    mean_evals = np.mean(function_evals)
    variance_fitness = np.var(best_fitnesses)
    success_rate = np.mean(np.array(best_fitnesses) <= 0.01)
    max_peak_ratio = np.clip(np.mean(np.abs(np.array(best_fitnesses))), 0, 1)
    peak_coverage = len(set(best_fitnesses)) / 10  # Assume 10 known peaks
    population_diversity = np.var(all_solutions)
    success_rate_threshold = np.mean(np.array(best_fitnesses) <= 0.05)
    convergence_speed = np.mean(function_evals) / max_iterations

    # Store metrics
    metrics["Function"].append(name)
    metrics["Local Optima"].append(0)
    metrics["Global Optima"].append(1)
    metrics["Average Function Evaluations"].append(mean_evals)
    metrics["LEA Average Best Fitness"].append(mean_fitness)
    metrics["Success Rate (SR)"].append(success_rate)
    metrics["Success Performance (SP)"].append(mean_evals)
    metrics["Maximum Peak Ratio (MPR)"].append(max_peak_ratio)
    metrics["Peak Coverage (PC)"].append(peak_coverage)
    metrics["Robustness Score"].append(variance_fitness)
    metrics["Population Diversity (PD)"].append(population_diversity)
    metrics["Success Rate with Threshold (SRT)"].append(success_rate_threshold)
    metrics["Convergence Speed (CS)"].append(convergence_speed)

# Store results in DataFrame
df_results = pd.DataFrame(metrics)

df_results.to_csv("LEA_Results.csv", index=False)
print(df_results)


<ipython-input-11-3c642657aa91>:17: RuntimeWarning: invalid value encountered in arccos
  return sum([(np.cos((j + 1) * np.arccos(x[i])) - (-1) ** j) ** 2 for i in range(n) for j in range(n)])


               Function  Local Optima  Global Optima  \
0     Storn’s Chebyshev             0              1   
1       Inverse Hilbert             0              1   
2         Lennard-Jones             0              1   
3             Rastrigin             0              1   
4              Griewank             0              1   
5           Weierstrass             0              1   
6     Modified Schwefel             0              1   
7  Expanded Schaffer F6             0              1   
8             Happy Cat             0              1   
9                Ackley             0              1   

   Average Function Evaluations  LEA Average Best Fitness  Success Rate (SR)  \
0                       15000.0                       NaN                0.0   
1                       15000.0              9.147797e+12                0.0   
2                       15000.0             -3.785021e-05                1.0   
3                       15000.0              8.876365e+03      

In [1]:
import numpy as np
import pandas as pd
from scipy.special import gamma

# Define Levy Flight Function
def levy_flight(d, beta=1.5):
    sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) /
             (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
    u = np.random.randn(1, d) * sigma
    v = np.random.randn(1, d)
    step = u / (np.abs(v) ** (1 / beta))
    return 0.01 * step.flatten()

# Define the benchmark functions
def storn_chebyshev(x):
    n = len(x)
    return sum([(np.cos((j + 1) * np.arccos(x[i])) - (-1) ** j) ** 2 for i in range(n) for j in range(n)])

def inverse_hilbert(x):
    n = len(x)
    H = np.array([[1 / (i + j + 1) for j in range(n)] for i in range(n)])
    return np.linalg.norm(np.linalg.inv(H) - np.diag(x))

def lennard_jones(x, epsilon=1e-9):
    n = len(x) // 3
    X, Y, Z = x[:n], x[n:2*n], x[2*n:]
    energy = 0
    for i in range(n):
        for j in range(i + 1, n):
            r2 = (X[i] - X[j])**2 + (Y[i] - Y[j])**2 + (Z[i] - Z[j])**2
            if r2 < epsilon:
                continue  # Avoid division by zero
            energy += 4 * ((1/r2)**6 - (1/r2)**3)
    return energy

def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

def griewank(x):
    sum_part = sum(xi**2 / 4000 for xi in x)
    prod_part = np.prod([np.cos(xi / np.sqrt(i + 1)) for i, xi in enumerate(x)])
    return sum_part - prod_part + 1

def weierstrass(x, a=0.5, b=3, k_max=20):
    return sum([sum([a**k * np.cos(2 * np.pi * b**k * (xi + 0.5)) for k in range(k_max)]) for xi in x])

def modified_schwefel(x):
    return 418.9829 * len(x) - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

def expanded_schaffer_f6(x):
    def f6(xi, xj):
        return 0.5 + (np.sin(np.sqrt(xi**2 + xj**2))**2 - 0.5) / (1 + 0.001 * (xi**2 + xj**2))**2
    return sum([f6(x[i], x[i+1]) for i in range(len(x)-1)])

def happy_cat(x):
    alpha = 1/8
    sum_x2 = sum([xi**2 for xi in x])
    return ((sum_x2 - len(x))**2)**alpha + (0.5 * sum_x2 + sum(x)) / len(x) + 0.5

def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

# Function details with search range and dimensions
function_details = {
    "Storn’s Chebyshev": (9, [-8192, 8192]),
    "Inverse Hilbert": (16, [-16384, 16384]),
    "Lennard-Jones": (18, [-4, 4]),
    "Rastrigin": (10, [-100, 100]),
    "Griewank": (10, [-100, 100]),
    "Weierstrass": (10, [-100, 100]),
    "Modified Schwefel": (10, [-100, 100]),
    "Expanded Schaffer F6": (10, [-100, 100]),
    "Happy Cat": (10, [-100, 100]),
    "Ackley": (10, [-100, 100])
}

benchmark_functions = {
    "Storn’s Chebyshev": storn_chebyshev,
    "Inverse Hilbert": inverse_hilbert,
    "Lennard-Jones": lennard_jones,
    "Rastrigin": rastrigin,
    "Griewank": griewank,
    "Weierstrass": weierstrass,
    "Modified Schwefel": modified_schwefel,
    "Expanded Schaffer F6": expanded_schaffer_f6,
    "Happy Cat": happy_cat,
    "Ackley": ackley
}

# Global Optima for Success Rate Calculation
global_optima = {
    "Storn’s Chebyshev": 0.0,
    "Inverse Hilbert": 0.0,
    "Lennard-Jones": -1.0,
    "Rastrigin": 0.0,
    "Griewank": 0.0,
    "Weierstrass": 0.0,
    "Modified Schwefel": 0.0,
    "Expanded Schaffer F6": 0.0,
    "Happy Cat": 0.0,
    "Ackley": 0.0
}


# Define the Improved LEA Algorithm
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_iterations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_iterations = max_iterations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.personal_best = self.population.copy()
        self.personal_best_fitness = np.array([self.fitness_function(ind) for ind in self.population])

    def optimize(self):
        best_fitnesses = []
        function_evaluations = 0

        for iter in range(self.max_iterations):
            # Adaptive parameters
            w = 0.9 * (1 - (iter / self.max_iterations) ** 2) + 0.4
            my_c = max(0.1 * np.exp(-iter / (self.max_iterations / 10)), 0)

            for i in range(self.population_size):
                # Adaptive Levy Flight scaling based on diversity
                diversity = np.var(self.population)
                levy_scale = 1 / (1 + np.exp(-diversity + 0.5))

                if np.random.rand() < 0.5:
                    self.population[i] += levy_flight(self.dimensions) * levy_scale * self.population[i]
                else:
                    self.population[i] += np.random.uniform(0.5, 1) * (self.personal_best[i] - self.population[i])

                # Apply opposition-based learning
                opposite_solution = self.lower_bound + self.upper_bound - self.population[i]
                if self.fitness_function(opposite_solution) < self.fitness_function(self.population[i]):
                    self.population[i] = opposite_solution

                # Boundary handling
                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
                fitness = self.fitness_function(self.population[i])

                # Update personal best
                if fitness < self.personal_best_fitness[i]:
                    self.personal_best[i] = self.population[i]
                    self.personal_best_fitness[i] = fitness

                best_fitnesses.append(fitness)
                function_evaluations += 1

            # Diversity-based reinitialization
            diversity_threshold = 1e-4
            if np.var(self.population) < diversity_threshold:
                self.population[np.random.randint(0, self.population_size, size=5)] = np.random.uniform(
                    self.lower_bound, self.upper_bound, (5, self.dimensions))

        return np.min(best_fitnesses), function_evaluations

# Run LEA Optimization and Collect Metrics
num_runs = 10
max_iterations = 500

metrics = {
    "Function": [],
    "Average Function Evaluations": [],
    "LEA Average Best Fitness": [],
    "Success Rate (SR)": [],
    "Robustness Score": [],
    "Population Diversity (PD)": [],
    "Success Rate with Threshold (SRT)": [],
    "Convergence Speed (CS)": []
}

for name, func in benchmark_functions.items():
    best_fitnesses, function_evals, all_solutions = [], [], []

    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(30, 10, -100, 100, max_iterations, func)
        best_fitness, function_evaluations = lea.optimize()
        best_fitnesses.append(best_fitness)
        function_evals.append(function_evaluations)
        all_solutions.append(best_fitness)

    mean_fitness = np.mean(best_fitnesses)
    mean_evals = np.mean(function_evals)
    variance_fitness = np.var(best_fitnesses)
    success_rate = np.mean(np.array(best_fitnesses) <= 0.01)
    population_diversity = np.var(all_solutions)
    success_rate_threshold = np.mean(np.array(best_fitnesses) <= 0.05)
    convergence_speed = np.mean(function_evals) / max_iterations

    # Store metrics
    metrics["Function"].append(name)
    metrics["Average Function Evaluations"].append(mean_evals)
    metrics["LEA Average Best Fitness"].append(mean_fitness)
    metrics["Success Rate (SR)"].append(success_rate)
    metrics["Robustness Score"].append(variance_fitness)
    metrics["Population Diversity (PD)"].append(population_diversity)
    metrics["Success Rate with Threshold (SRT)"].append(success_rate_threshold)
    metrics["Convergence Speed (CS)"].append(convergence_speed)
# Store results in DataFrame
df_results = pd.DataFrame(metrics)

df_results.to_csv("NewLEA_Results.csv", index=False)
print(df_results)





<ipython-input-1-3a6e9d2da1a8>:17: RuntimeWarning: invalid value encountered in arccos
  return sum([(np.cos((j + 1) * np.arccos(x[i])) - (-1) ** j) ** 2 for i in range(n) for j in range(n)])


               Function  Average Function Evaluations  \
0     Storn’s Chebyshev                       15000.0   
1       Inverse Hilbert                       15000.0   
2         Lennard-Jones                       15000.0   
3             Rastrigin                       15000.0   
4              Griewank                       15000.0   
5           Weierstrass                       15000.0   
6     Modified Schwefel                       15000.0   
7  Expanded Schaffer F6                       15000.0   
8             Happy Cat                       15000.0   
9                Ackley                       15000.0   

   LEA Average Best Fitness  Success Rate (SR)  Robustness Score  \
0                       NaN                0.0               NaN   
1              9.147797e+12                0.0      2.378348e+01   
2             -9.999987e-01                1.0      1.015184e-11   
3              3.046535e+03                0.0      5.270817e+05   
4              1.888267e+00     

In [3]:
import numpy as np
import pandas as pd
from scipy.special import gamma
from scipy.stats import friedmanchisquare, wilcoxon

# Define Levy Flight Function (for LEA)
def levy_flight(d, beta=1.5):
    sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) /
             (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
    u = np.random.randn(1, d) * sigma
    v = np.random.randn(1, d)
    step = u / (np.abs(v) ** (1 / beta))
    return 0.01 * step.flatten()

# Benchmark Functions (same as LEA)
def storn_chebyshev(x):
    n = len(x)
    return sum([(np.cos((j + 1) * np.arccos(x[i])) - (-1) ** j) ** 2 for i in range(n) for j in range(n)])

def inverse_hilbert(x):
    n = len(x)
    H = np.array([[1 / (i + j + 1) for j in range(n)] for i in range(n)])
    return np.linalg.norm(np.linalg.inv(H) - np.diag(x))

def lennard_jones(x, epsilon=1e-9):
    n = len(x) // 3
    X, Y, Z = x[:n], x[n:2*n], x[2*n:]
    energy = 0
    for i in range(n):
        for j in range(i + 1, n):
            r2 = (X[i] - X[j])**2 + (Y[i] - Y[j])**2 + (Z[i] - Z[j])**2
            if r2 < epsilon:
                continue
            energy += 4 * ((1/r2)**6 - (1/r2)**3)
    return energy

def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

def griewank(x):
    sum_part = sum(xi**2 / 4000 for xi in x)
    prod_part = np.prod([np.cos(xi / np.sqrt(i + 1)) for i, xi in enumerate(x)])
    return sum_part - prod_part + 1

def weierstrass(x, a=0.5, b=3, k_max=20):
    return sum([sum([a**k * np.cos(2 * np.pi * b**k * (xi + 0.5)) for k in range(k_max)]) for xi in x])

def modified_schwefel(x):
    return 418.9829 * len(x) - sum([xi * np.sin(np.sqrt(abs(xi))) for xi in x])

def expanded_schaffer_f6(x):
    def f6(xi, xj):
        return 0.5 + (np.sin(np.sqrt(xi**2 + xj**2))**2 - 0.5) / (1 + 0.001 * (xi**2 + xj**2))**2
    return sum([f6(x[i], x[i+1]) for i in range(len(x)-1)])

def happy_cat(x):
    alpha = 1/8
    sum_x2 = sum([xi**2 for xi in x])
    return ((sum_x2 - len(x))**2)**alpha + (0.5 * sum_x2 + sum(x)) / len(x) + 0.5

def ackley(x):
    a, b, c = 20, 0.2, 2 * np.pi
    d = len(x)
    return -a * np.exp(-b * np.sqrt(np.sum(x**2) / d)) - np.exp(np.sum(np.cos(c * x)) / d) + a + np.exp(1)

# Function details
function_details = {
    "Storn’s Chebyshev": (9, [-1, 1]),
    "Inverse Hilbert": (16, [-16384, 16384]),
    "Lennard-Jones": (18, [-4, 4]),
    "Rastrigin": (10, [-100, 100]),
    "Griewank": (10, [-100, 100]),
    "Weierstrass": (10, [-100, 100]),
    "Modified Schwefel": (10, [-100, 100]),
    "Expanded Schaffer F6": (10, [-100, 100]),
    "Happy Cat": (10, [-100, 100]),
    "Ackley": (10, [-100, 100])
}

benchmark_functions = {
    "Storn’s Chebyshev": storn_chebyshev,
    "Inverse Hilbert": inverse_hilbert,
    "Lennard-Jones": lennard_jones,
    "Rastrigin": rastrigin,
    "Griewank": griewank,
    "Weierstrass": weierstrass,
    "Modified Schwefel": modified_schwefel,
    "Expanded Schaffer F6": expanded_schaffer_f6,
    "Happy Cat": happy_cat,
    "Ackley": ackley
}

# Lotus Effect Algorithm (LEA)
class LotusEffectAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_iterations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_iterations = max_iterations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.personal_best = self.population.copy()
        self.personal_best_fitness = np.array([self.fitness_function(ind) for ind in self.population])

    def optimize(self):
        best_fitnesses = []
        function_evaluations = 0

        for iter in range(self.max_iterations):
            w = 0.9 * (1 - (iter / self.max_iterations) ** 2) + 0.4
            my_c = max(0.1 * np.exp(-iter / (self.max_iterations / 10)), 0)

            for i in range(self.population_size):
                diversity = np.var(self.population)
                levy_scale = 1 / (1 + np.exp(-diversity + 0.5))

                if np.random.rand() < 0.5:
                    self.population[i] += levy_flight(self.dimensions) * levy_scale * self.population[i]
                else:
                    self.population[i] += np.random.uniform(0.5, 1) * (self.personal_best[i] - self.population[i])

                opposite_solution = self.lower_bound + self.upper_bound - self.population[i]
                if self.fitness_function(opposite_solution) < self.fitness_function(self.population[i]):
                    self.population[i] = opposite_solution

                self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
                fitness = self.fitness_function(self.population[i])

                if fitness < self.personal_best_fitness[i]:
                    self.personal_best[i] = self.population[i]
                    self.personal_best_fitness[i] = fitness

                best_fitnesses.append(fitness)
                function_evaluations += 1

            diversity_threshold = 1e-4
            if np.var(self.population) < diversity_threshold:
                self.population[np.random.randint(0, self.population_size, size=5)] = np.random.uniform(
                    self.lower_bound, self.upper_bound, (5, self.dimensions))

        return np.min(best_fitnesses), function_evaluations, best_fitnesses

# Multi-modal Battle Royale Optimization (MBRO)
class MultiModalBattleRoyale:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_iterations, fitness_function):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_iterations = max_iterations
        self.fitness_function = fitness_function
        self.population = np.random.uniform(lower_bound, upper_bound, (population_size, dimensions))
        self.fitness = np.array([self.fitness_function(ind) for ind in self.population])
        self.damage = np.zeros(population_size)  # Damage level for each solution
        self.threshold = 5  # Damage threshold from paper (T=5)
        self.w_max, self.w_min = 0.4, 0.0001  # Step size bounds

    def optimize(self):
        best_fitnesses = []
        function_evaluations = 0

        for iter in range(self.max_iterations):
            # Calculate step size w (Eq. 7)
            w = self.w_max - ((self.w_max - self.w_min) / self.max_iterations) * iter

            for i in range(self.population_size):
                # Compute electrostatic forces using Coulomb's law (Eq. 5)
                forces = []
                for j in range(self.population_size):
                    if i != j:
                        r2 = np.sum((self.population[i] - self.population[j])**2)
                        if r2 == 0:  # Avoid division by zero
                            r2 = 1e-10
                        force = (self.fitness[i] * self.fitness[j]) / r2  # alpha = 1
                        forces.append((force, j))
                if forces:
                    _, neighbor_idx = max(forces, key=lambda x: x[0])  # argmax(F)
                    neighbor_fitness = self.fitness[neighbor_idx]

                    # Compare with "fittest-and-closest" neighbor
                    if self.fitness[i] < neighbor_fitness:
                        self.damage[i] += 1  # Increase damage if current is better
                    else:
                        self.damage[neighbor_idx] += 1

                    # Movement based on damage level
                    if self.damage[i] >= self.threshold:
                        # Move toward winner (Eq. 2 adapted)
                        r = np.random.rand()
                        self.population[i] = self.population[i] + r * (self.population[neighbor_idx] - self.population[i])
                        self.damage[i] = 0  # Reset damage
                    else:
                        # Run-and-tumble motion (Eq. 6)
                        delta = (self.upper_bound - self.lower_bound) * np.random.rand(self.dimensions) + self.lower_bound
                        lambda_param = w  # Use w as control parameter
                        self.population[i] += lambda_param * delta

                    # Boundary check
                    self.population[i] = np.clip(self.population[i], self.lower_bound, self.upper_bound)
                    self.fitness[i] = self.fitness_function(self.population[i])
                    function_evaluations += 1
                    best_fitnesses.append(self.fitness[i])

        return np.min(best_fitnesses), function_evaluations, best_fitnesses

# Run Comparison
num_runs = 10
max_iterations = 500

metrics = {
    "Function": [],
    "LEA Avg Best Fitness": [], "LEA Success Rate": [], "LEA Avg Evals": [],
    "MBRO Avg Best Fitness": [], "MBRO Success Rate": [], "MBRO Avg Evals": []
}

lea_results = {name: [] for name in benchmark_functions.keys()}
mbro_results = {name: [] for name in benchmark_functions.keys()}

for name, func in benchmark_functions.items():
    dimensions, (lower_bound, upper_bound) = function_details[name]

    # LEA Results
    lea_best_fitnesses, lea_evals = [], []
    for _ in range(num_runs):
        lea = LotusEffectAlgorithm(30, dimensions, lower_bound, upper_bound, max_iterations, func)
        best_fitness, evals, run_fitnesses = lea.optimize()
        lea_best_fitnesses.append(best_fitness)
        lea_evals.append(evals)
        lea_results[name].append(min(run_fitnesses))

    # MBRO Results
    mbro_best_fitnesses, mbro_evals = [], []
    for _ in range(num_runs):
        mbro = MultiModalBattleRoyale(30, dimensions, lower_bound, upper_bound, max_iterations, func)
        best_fitness, evals, run_fitnesses = mbro.optimize()
        mbro_best_fitnesses.append(best_fitness)
        mbro_evals.append(evals)
        mbro_results[name].append(min(run_fitnesses))

    # Metrics
    metrics["Function"].append(name)
    metrics["LEA Avg Best Fitness"].append(np.mean(lea_best_fitnesses))
    metrics["LEA Success Rate"].append(np.mean(np.array(lea_best_fitnesses) <= 0.01))
    metrics["LEA Avg Evals"].append(np.mean(lea_evals))
    metrics["MBRO Avg Best Fitness"].append(np.mean(mbro_best_fitnesses))
    metrics["MBRO Success Rate"].append(np.mean(np.array(mbro_best_fitnesses) <= 0.01))
    metrics["MBRO Avg Evals"].append(np.mean(mbro_evals))

# Results DataFrame
df_results = pd.DataFrame(metrics)
df_results.to_csv("LEA_vs_MBRO_Results.csv", index=False)
print(df_results)

# Statistical Tests
print("\nPerforming Friedman Test:")
lea_array = np.array([lea_results[name] for name in benchmark_functions.keys()])
mbro_array = np.array([mbro_results[name] for name in benchmark_functions.keys()])
combined_array = np.vstack((lea_array, mbro_array))
statistic, p_value = friedmanchisquare(*combined_array)
print(f"Friedman Test Statistic: {statistic:.4f}")
print(f"p-value: {p_value:.4f}")
print("Interpretation: If p-value < 0.05, there are significant differences between algorithms")

print("\nPerforming Wilcoxon Signed-Rank Tests:")
for name in benchmark_functions.keys():
    try:
        stat, p_val = wilcoxon(lea_results[name], mbro_results[name])
        print(f"\n{name}: LEA vs MBRO")
        print(f"Wilcoxon Statistic: {stat:.4f}")
        print(f"p-value: {p_val:.4f}")
        print("Interpretation: If p-value < 0.05, there is a significant difference")
    except ValueError as e:
        print(f"\n{name}: LEA vs MBRO")
        print(f"Error: {e}")

<ipython-input-3-6dcbe04e8e19>:18: RuntimeWarning: invalid value encountered in arccos
  return sum([(np.cos((j + 1) * np.arccos(x[i])) - (-1) ** j) ** 2 for i in range(n) for j in range(n)])


               Function  LEA Avg Best Fitness  LEA Success Rate  \
0     Storn’s Chebyshev          9.077004e+01               0.0   
1       Inverse Hilbert          1.697759e+18               0.0   
2         Lennard-Jones         -3.594153e+00               1.0   
3             Rastrigin          3.465041e+03               0.0   
4              Griewank          1.887909e+00               0.0   
5           Weierstrass         -1.999998e+01               1.0   
6     Modified Schwefel          3.654157e+03               0.0   
7  Expanded Schaffer F6          3.381317e+00               0.0   
8             Happy Cat          3.486114e+00               0.0   
9                Ackley          2.004673e+01               0.0   

   LEA Avg Evals  MBRO Avg Best Fitness  MBRO Success Rate  MBRO Avg Evals  
0        15000.0           9.277247e+01                0.0         15000.0  
1        15000.0           1.697759e+18                0.0         15000.0  
2        15000.0          -4.61